In [18]:
import math

In [47]:
def f(x):
    """ f(x) = x + lg(x) - 0.5 """
    return x + math.log10(x) - 0.5

In [49]:
def numerical_derivative(func, x, h=1e-5):
    return (func(x + h) - func(x - h)) / (2 * h)

In [59]:
def configure_phi(a, b):
    steps = 100
    step_val = (b - a) / steps
    
    max_deriv = -float('inf')
    
    for i in range(steps + 1):
        x_val = a + i * step_val
            
        deriv = numerical_derivative(f, x_val)
        
        if deriv > max_deriv:
            max_deriv = deriv

    if max_deriv == 0:
        raise ValueError("Похідна дорівнює 0, метод неможливий.")
        
    m = 1.0 / max_deriv
    
    print(f"   Макс. похідна: {max_deriv:.4f}")
    print(f"   Обчислено параметр m: {m:.4f}")
    print(f"   Формула ітерації: x_{{k+1}} = x_k - {m:.4f} * f(x_k)")


    def phi_dynamic(x):
        # Формула: x = x - m * f(x)
        return x - m * f(x)
    
    return phi_dynamic

In [61]:
def get_automatic_params(left=-10000, right=10000, interval_threshold=1):
    
    if left <= 0:
        actual_left = 0.0001

    else:
        actual_left = left
        
    actual_right = right
    print(f"--> Запуск бінарного пошуку в межах [{actual_left}, {actual_right}]...")

    try:
        y_left = f(actual_left)
        y_right = f(actual_right)
    except ValueError:
        print("Помилка обчислення функції на межах інтервалу.")
        return None

    if y_left * y_right > 0:
        print(f"Помилка: На інтервалі [{actual_left}, {actual_right}] функція не змінює знак.")
        print("Корінь відсутній або інтервал задано неправильно.")
        return None
    
    iteration = 0
    while (actual_right - actual_left) > interval_threshold:
        mid = (actual_left + actual_right) / 2
        
        try:
            y_mid = f(mid)
            y_left_val = f(actual_left)
            
            if y_left_val * y_mid < 0:
                actual_right = mid
            else:
                actual_left = mid
                
        except ValueError:
            actual_left = mid  
        iteration += 1


    x0 = (actual_left + actual_right) / 2
    
    print(f"--> Інтервал знайдено за {iteration} кроків бісекції.")
    return actual_left, actual_right, x0

In [63]:
def solve_simple_iteration(phi_func, x, epsilon):
    k = 0
    print("-" * 65)
    print(f"{'k':<5} {'x':<20} {'phi(x)':<20} {'delta':<20}")
    print("-" * 65)
    
    while True:
        y = phi_func(x)
        delta = abs(y - x)
        
        print(f"{k:<5} {x:<20.15f} {y:<20.15f} {delta:<20.6e}")
        
        if delta < epsilon:
            return y, k
        k += 1
        
        x = y


In [65]:
def main(epsilon = 1e-15):
    
    params = get_automatic_params()
    
    if params is None:
        print("Помилка: Корінь не знайдено в заданому діапазоні.")
        return
        
    a, b, x0 = params
    print(f"\n a  = {a}\n b  = {b}\n x0 = {x0}\n")
    
    root, iterations = solve_simple_iteration(configure_phi(a, b), x0, epsilon)
    
    print("-" * 65)
    print(f"Корінь: {root:.16f}")


    residual = root + math.log10(root) - 0.5
    try:
        with open("results_main.txt", "w", encoding="utf-8") as file:
            file.write("Рівняння: x + lg(x) = 0.5\n\n")
            file.write("Вхідні дані:\n")
            file.write(f"  Інтервал [a, b]:  [{a}, {b}]\n")
            file.write(f"  Початкове x0:     {x0}\n")
            file.write(f"  Точність (eps):   {epsilon}\n\n")
            
            file.write("Результат:\n")
            file.write(f"  Знайдений корінь: {root:.15f}\n")
            file.write(f"  Кількість ітерацій: {iterations}\n")
            file.write(f"  Фактична похибка (f(x)): {residual:.5e}\n")     
    except IOError as e:
        print(f"Помилка при записі файлу: {e}")
    

In [67]:
main()

--> Запуск бінарного пошуку в межах [0.0001, 10000]...
--> Інтервал знайдено за 14 кроків бісекції.

 a  = 0.6104515563964844
 b  = 1.2208031127929688
 x0 = 0.9156273345947266

   Макс. похідна: 1.7114
   Обчислено параметр m: 0.5843
   Формула ітерації: x_{k+1} = x_k - 0.5843 * f(x_k)
-----------------------------------------------------------------
k     x                    phi(x)               delta               
-----------------------------------------------------------------
0     0.915627334594727    0.695141689470026    2.204856e-01        
1     0.695141689470026    0.673396721010720    2.174497e-02        
2     0.673396721010720    0.672422262891261    9.744581e-04        
3     0.672422262891261    0.672384664697097    3.759819e-05        
4     0.672384664697097    0.672383224687773    1.440009e-06        
5     0.672383224687773    0.672383169551250    5.513652e-08        
6     0.672383169551250    0.672383167440150    2.111099e-09        
7     0.672383167440150    0.